In [ ]:
import polars as pl
from datetime import datetime,timedelta
import get_and_store as gas
import polars_indicators as pi
from polars_indicators.indicators import IndicatorResult
from importlib import reload
DATE_COLUNN = "Date"

In [ ]:
reload(pi)
reload(pi.strategies.buy_x_week_low)
reload(pi.summarize.trades)
reload(pi.indicators)

In [ ]:
symbols = ['A','AA']

In [ ]:
#df = gas.get_symbols_df(symbols)

In [ ]:
try:
    df = saved_df.clone()
except NameError:
        df = gas.get_all_saved_data_df()
        df = df.filter(pl.col(pi.indicators.DATE_COLUMN) >= datetime(2020, 1, 1))
        df = df.collect()

In [ ]:
#this date was missing before
#df.filter(pl.col(pi.indicators.DATE_COLUMN) == datetime(2020,4,2))

In [ ]:
saved_df = df.clone()

In [ ]:
fluctuation = pi.indicators.fluctuation_percentage(df)
fluct_sma3 = pi.indicators.simple_moving_average(fluctuation.df, 7, fluctuation.column)
#grouped_fluctation = pi.indicators.group_by_amount_display(fluct_sma3.df, fluct_sma3.column, [1,2,3,5,7,10,20,30,40,50,60,70,80,90])
grouped_fluctation = pi.indicators.group_by_amount(fluct_sma3.df, fluct_sma3.column, [1,2,3,5,7,10,20])
df = grouped_fluctation.df

In [ ]:
relative_volume = pi.indicators.relative_volume(df, 5)
grouped_relative_volume = pi.indicators.group_by_amount(relative_volume.df, relative_volume.column, [x * 25 for x in range(1,7)])
df = grouped_relative_volume.df

In [ ]:
df = df.with_columns(pl.col(pi.indicators.DATE_COLUMN).cast(pl.Datetime))
df

In [ ]:
#filter out average price too low
df = df.filter(pl.col(pi.indicators.CLOSE_COLUMN).mean().over(pi.indicators.SYMBOL_COLUMN) > 3)

# filter out too many 0 volume bars. This gets rid of a lot of garbage
df = df.filter((pl.col(pi.indicators.VOLUME_COLUMN) == 0).sum().over(pi.indicators.SYMBOL_COLUMN) < 3)

# minimum amount of volume
df = df.filter(pl.col(pi.indicators.VOLUME_COLUMN).mean().over(pi.indicators.SYMBOL_COLUMN) > 100000)


In [ ]:
lookback_bars = 7
offset_bars = 3
modresult_05 = pi.strategies.buy_x_week_low.strategy(df, lookback_bars, offset_bars, 0.5, -5, 2, 3)
modresult_1 = pi.strategies.buy_x_week_low.strategy(df, lookback_bars, offset_bars, 1, -5, 2, 3)

In [ ]:
extra_columns = [grouped_fluctation.column, grouped_relative_volume.column]
trades_05 = pi.summarize.trades.Trades.from_strategy_result(modresult_05, extra_columns)
trades_1 = pi.summarize.trades.Trades.from_strategy_result(modresult_1, extra_columns)

In [ ]:
trades_1.summarize_strategy(extra_columns).sort(grouped_fluctation.column).filter((
    (pl.col(grouped_fluctation.column) >= 10) & (pl.col(grouped_relative_volume.column) == 25))
    |
    (pl.col(grouped_fluctation.column) == 20.01) & (pl.col(grouped_relative_volume.column) == 50))

In [ ]:
res = pi.strategies.buy_x_week_low.fluctuation_and_relative_volume(df, 7, 5)
pi.summarize.trades.Trades.from_strategy_result(res, extra_columns).summarize_strategy(extra_columns)